In [1]:
import itertools
import pandas as pd
import re
from log_class import log_recorder
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np

import random
import math
import time

LOG_DIR = "./test_data2/"
report_csv = "report.csv"
LOG_file = "LOG"


# from log_class import log_recorder
COMPACTION_LOG_HEAD = "/compaction/compaction_job.cc:755"
FLUSH_LOG_BEGIN = "flush_started"
FLUSH_LOG_END = "flush_finished"
FLUSH_FILE_CREATEION = "table_file_creation"

def load_log_and_qps(log_file, ground_truth_csv):
    # load the data
    return log_recorder(log_file,ground_truth_csv)

data_set = load_log_and_qps(LOG_DIR+LOG_file, LOG_DIR+report_csv)

ModuleNotFoundError: No module named 'spacy'

# Cut the timeline into pieces according to the time slice size

In [2]:
ms_to_sec  = 1000000
time_slice = 100000 # 100 miusec, 100,000ms
switch_ratio = ms_to_sec/time_slice

real_time_speed=data_set.qps_df
flush_jobs = data_set.flush_df
# op_type, 0 for flush, 1 for compaction; 
# triggered or not, 0 for not triggered, 1 for running
# input size, how many bytes have been read from disk
# total_operation_bytes
# job_id, i don't kown whether it can be used or not.
input_tuple=[0,0,12345,7] 

# then we put all the operations into a series of input tupler stack
#        -----
#      ----
# --- -- 
# -- -------
# 1 2 3 4 5 6 7 8 9 10



# create the bucket first

# print(int(flush_jobs[flush_jobs["job"]==11902]["start_time"]/time_slice))


NameError: name 'data_set' is not defined

In [3]:

bucket = []
for i in range(int(real_time_speed.tail(1)["microsecs_elapsed"] * switch_ratio)):
    bucket.append([])
# then we use a bucket sort idea to count down the rest things
for flush_job in data_set.flush_df.iloc():
    # indices = (int(flush_job["start_time"]/time_slice), flush_job["end_time"]/time_slice)
    start_index = int(flush_job["start_time"]/time_slice)
    end_index=int(flush_job["end_time"]/time_slice) + 1
    payload = round(flush_job["flush_size"] / (1024*1024) ,2) # change to MB will be easier to calculate
    job_id = flush_job["job"]

    if start_index >= len(bucket)-10 or end_index >= len(bucket)-5: # the tail part is not accurant
        break
    for bucket_element in bucket[start_index:end_index]:
        bucket_element.append([0,0,payload,job_id])


for compaction_job in data_set.compaction_df.iloc():
    start_index = int(compaction_job["start_time"]/time_slice)
    end_index=int(compaction_job["end_time"]/time_slice) + 1
    input_size = round(compaction_job["input_data_size"] / (1024*1024) ,2) # change to MB will be easier to calculate
    output_size = round(compaction_job["total_output_size"] / (1024*1024) ,2)
    job_id = compaction_job["job"]
    if start_index >= len(bucket)-10 or end_index >= len(bucket)-5: # the tail part is not accurant
        break
    compaction_tuple=[1,input_size,output_size,job_id]
    for bucket_element in bucket[start_index:end_index]:
        bucket_element.append(compaction_tuple)

result_tensor = real_time_speed["interval_qps"]
max_length = max([len(x) for x in bucket])

NameError: name 'real_time_speed' is not defined

In [4]:

empty_tuple = [0,0,0,0]
for element in bucket:
    while len(element) < max_length:
        element.append(empty_tuple)
import torch
import numpy as np 

In [5]:
input_tensor = torch.tensor(bucket)
qps_list = list(data_set.qps_df["interval_qps"]) 
while len(qps_list) < input_tensor.size(0):
    qps_list.append(0)

output_tensor = torch.tensor(qps_list).reshape(-1,1)

print(input_tensor.size(),output_tensor.size())

NameError: name 'data_set' is not defined

In [6]:
flatten_input = []
for element in bucket:
    temp_list = []
    for xx in element:
        for x in xx:
            temp_list.append(x)
    flatten_input.append(temp_list)

flatten_input_tensor = torch.tensor(flatten_input)
# # print(flatten_input)
print(flatten_input_tensor.size())


torch.Size([0])


In [7]:
time_idx = set(data_set.qps_df["microsecs_elapsed"])
aggregated_qps = []
for time_id in time_idx:
    aggregated_qps.append(sum(data_set.qps_df["interval_qps"][time_id == data_set.qps_df["microsecs_elapsed"]]))

aggregate_output_tensor = torch.tensor(aggregated_qps).reshape(-1,1)

NameError: name 'data_set' is not defined

In [8]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src